0 导入包

In [1]:
from datasets import Dataset
from datasets import load_dataset
from datasets import DatasetDict

1 加载数据集

In [2]:
dataset_dict = load_dataset('m-a-p/COIG-CQIA', 'ruozhiba')
dataset = dataset_dict['train']
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'task_type', 'domain', 'metadata', 'answer_from', 'human_verified', 'copyright'],
        num_rows: 240
    })
})

2 dataset -> df

In [3]:
# dataset -> dataframe
df = dataset.to_pandas()
'''dataframe不能够处理嵌套格式的数据集，因此转换会造成丢失
形如 a :{{b:200}，{c:100}}不能被处理
'''

'dataframe不能够处理嵌套格式的数据集，因此转换会造成丢失\n形如 a :{{b:200}，{c:100}}不能被处理\n'

3 df -> dataset

In [4]:
# df -> dataset
dataset = Dataset.from_pandas(df) 
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'task_type', 'domain', 'metadata', 'answer_from', 'human_verified', 'copyright'],
    num_rows: 240
})

4 重命名

In [5]:
dataset = dataset.rename_columns({'input': 'input2', 'output': 'output2'})
dataset

Dataset({
    features: ['instruction', 'input2', 'output2', 'task_type', 'domain', 'metadata', 'answer_from', 'human_verified', 'copyright'],
    num_rows: 240
})

5 删除列

In [6]:
dataset = dataset.remove_columns(['copyright','human_verified','answer_from'])
dataset

Dataset({
    features: ['instruction', 'input2', 'output2', 'task_type', 'domain', 'metadata'],
    num_rows: 240
})

6 筛选列

In [7]:
dataset = dataset.select_columns([ "input2","output2" ])
dataset

Dataset({
    features: ['input2', 'output2'],
    num_rows: 240
})

7 筛选数据数量

In [8]:
train_data = dataset.shuffle(seed=42).select(range(20)) # 随机选择前n条数据
train_data

Dataset({
    features: ['input2', 'output2'],
    num_rows: 20
})

8 切分数据集

In [9]:
# 切分为train，和test数据集
datasetdict = dataset.train_test_split(train_size=0.8, test_size=0.2, seed=42, shuffle=True)
datasetdict

DatasetDict({
    train: Dataset({
        features: ['input2', 'output2'],
        num_rows: 192
    })
    test: Dataset({
        features: ['input2', 'output2'],
        num_rows: 48
    })
})

9 构建数据集字典

In [10]:
datasetdict = DatasetDict() 
datasetdict['train'] = dataset.shuffle(seed=42).select(range(20)) # 随机选择前n条数据
datasetdict['test'] = dataset.shuffle(seed=42).select(range(20)) # 随机选择前n条数据
datasetdict

DatasetDict({
    train: Dataset({
        features: ['input2', 'output2'],
        num_rows: 20
    })
    test: Dataset({
        features: ['input2', 'output2'],
        num_rows: 20
    })
})

10 保存数据集

In [11]:
dataset.save_to_disk('save_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]

11 查看数据集特征

In [12]:
dataset.features 

{'input2': Value(dtype='string', id=None),
 'output2': Value(dtype='string', id=None)}

12 map函数

In [13]:
# 如果是datasetdict会自动分别处理，不需要做特殊改变
dataset = dataset.map(
    lambda x: # for i in dataset , x=dataset[i]
    {'a':x['output2']}, # 返回必须是字典格式，会字典形成新列表
    remove_columns=['input2', 'output2'], # 删除哪些列
    batched=True
)
dataset

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Dataset({
    features: ['a'],
    num_rows: 240
})

13 fliter函数

In [14]:
dataset = dataset.filter(
    lambda x: # for i in dataset , x=dataset[i]
    len(x["a"]) < 30, # 筛选符合要求的留下
)
dataset

Filter:   0%|          | 0/240 [00:00<?, ? examples/s]

Dataset({
    features: ['a'],
    num_rows: 2
})